In [1]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("bert-base-cased")
example="My name is Sylvain and I work at Hugging Face in Brooklyn."
encoding=tokenizer(example)
print(type(encoding))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [2]:
#to check if our tokenizer is fast or not
tokenizer.is_fast

True

In [4]:
encoding.tokens()

['[CLS]',
 'My',
 'name',
 'is',
 'S',
 '##yl',
 '##va',
 '##in',
 'and',
 'I',
 'work',
 'at',
 'Hu',
 '##gging',
 'Face',
 'in',
 'Brooklyn',
 '.',
 '[SEP]']

In [5]:
encoding.word_ids()

[None, 0, 1, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, None]

In [9]:
start,end=encoding.word_to_chars(3)
example[start:end]

'Sylvain'

In [11]:
#Inside token classififcation pipeline
from transformers import pipeline
token_classifier=pipeline("token-classification")
token_classifier("My name is Davitya and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-PER',
  'score': 0.99701893,
  'index': 4,
  'word': 'Da',
  'start': 11,
  'end': 13},
 {'entity': 'I-PER',
  'score': 0.9848393,
  'index': 5,
  'word': '##vity',
  'start': 13,
  'end': 17},
 {'entity': 'I-PER',
  'score': 0.99675554,
  'index': 6,
  'word': '##a',
  'start': 17,
  'end': 18},
 {'entity': 'I-ORG',
  'score': 0.96651834,
  'index': 11,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.97666055,
  'index': 12,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.98775,
  'index': 13,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9933444,
  'index': 15,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [13]:
token_classifier=pipeline("token-classification",aggregation_strategy="simple")# The aggregation_strategy picked will change the scores computed for each grouped entity. With "simple" the score is just the mean of the scores of each token in the given entity
token_classifier("My name is Davitya and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity_group': 'PER',
  'score': 0.9928713,
  'word': 'Davitya',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.97697634,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9933444,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [15]:
#from inputs to prediction
from transformers import AutoTokenizer,TFAutoModelForTokenClassification

pre_model="dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer=AutoTokenizer.from_pretrained(pre_model)
model=TFAutoModelForTokenClassification.from_pretrained(pre_model)
inputs=tokenizer(example,return_tensors="tf")
outputs=model(inputs)

All PyTorch model weights were used when initializing TFBertForTokenClassification.

All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


In [16]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)

(1, 19)
(1, 19, 9)


In [17]:
#in above output we can see that we have 1 sequence of 19 tokens and the model has 9 different labels

In [25]:
#to provide softmax function to normlize the logits to probabilities and to get predictions

import tensorflow as tf
probabilities=tf.math.softmax(outputs.logits,axis=-1)[0]
probabilities=probabilities.numpy().tolist()
predictions=tf.math.argmax(outputs.logits,axis=-1)[0]
predictions=predictions.numpy().tolist()
print(predictions)
print(probabilities)

[0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 6, 6, 6, 0, 8, 0, 0]
[[0.9994322061538696, 1.6470285117975436e-05, 3.4267035516677424e-05, 1.6042313291109167e-05, 8.250691462308168e-05, 2.138227500836365e-05, 0.00015649088891223073, 1.9652095943456516e-05, 0.00022089220874477178], [0.9989631175994873, 1.851577326306142e-05, 5.240462633082643e-05, 1.253474511031527e-05, 0.00043473768164403737, 3.087438381044194e-05, 0.0003146878443658352, 2.7860780392074957e-05, 0.00014510893379338086], [0.9997084140777588, 8.308127689815592e-06, 2.874564415833447e-05, 5.6503645282646175e-06, 8.694856660440564e-05, 9.783467248780653e-06, 6.78614669595845e-05, 1.1794005331466906e-05, 7.241901039378718e-05], [0.9998350143432617, 5.645536930387607e-06, 1.3955179383629002e-05, 4.313381850806763e-06, 4.017698665848002e-05, 8.123078259814065e-06, 5.648501610266976e-05, 8.991642971523106e-06, 2.723914076341316e-05], [0.00018333389016333967, 2.5156619813060388e-05, 4.846203955821693e-05, 1.490058366471203e-05, 0.999382913

In [21]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

In [28]:
results=[]
tokens=inputs.tokens()

for idx,pred in enumerate(predictions):
    label=model.config.id2label[pred]
    if label !="O":
        results.append({"entity": label,"score": probabilities[idx][pred],"word": tokens[idx]})
print(results)

[{'entity': 'I-PER', 'score': 0.9993829131126404, 'word': 'S'}, {'entity': 'I-PER', 'score': 0.998154878616333, 'word': '##yl'}, {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va'}, {'entity': 'I-PER', 'score': 0.9992326498031616, 'word': '##in'}, {'entity': 'I-ORG', 'score': 0.9738930463790894, 'word': 'Hu'}, {'entity': 'I-ORG', 'score': 0.9761150479316711, 'word': '##gging'}, {'entity': 'I-ORG', 'score': 0.9887974858283997, 'word': 'Face'}, {'entity': 'I-LOC', 'score': 0.9932106137275696, 'word': 'Brooklyn'}]


In [30]:
inputs_with_offsets=tokenizer(example,return_offsets_mapping=True)
inputs_with_offsets["offset_mapping"]

[(0, 0),
 (0, 2),
 (3, 7),
 (8, 10),
 (11, 12),
 (12, 14),
 (14, 16),
 (16, 18),
 (19, 22),
 (23, 24),
 (25, 29),
 (30, 32),
 (33, 35),
 (35, 40),
 (41, 45),
 (46, 48),
 (49, 57),
 (57, 58),
 (0, 0)]

In [32]:
results=[]
inputs_with_offsets=tokenizer(example,return_offsets_mapping=True)
tokens=inputs_with_offsets.tokens()
offsets=inputs_with_offsets["offset_mapping"]

for idx,pred in enumerate(predictions):
    label=model.config.id2label[pred]
    if label != "O":
      start,end=offsets[idx]
      results.append({"entity": label, "score": probabilities[idx][pred], "word": tokens[idx],"start":start,"end":end})
print(results)

[{'entity': 'I-PER', 'score': 0.9993829131126404, 'word': 'S', 'start': 11, 'end': 12}, {'entity': 'I-PER', 'score': 0.998154878616333, 'word': '##yl', 'start': 12, 'end': 14}, {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va', 'start': 14, 'end': 16}, {'entity': 'I-PER', 'score': 0.9992326498031616, 'word': '##in', 'start': 16, 'end': 18}, {'entity': 'I-ORG', 'score': 0.9738930463790894, 'word': 'Hu', 'start': 33, 'end': 35}, {'entity': 'I-ORG', 'score': 0.9761150479316711, 'word': '##gging', 'start': 35, 'end': 40}, {'entity': 'I-ORG', 'score': 0.9887974858283997, 'word': 'Face', 'start': 41, 'end': 45}, {'entity': 'I-LOC', 'score': 0.9932106137275696, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


In [ ]:
#Inside Question Answers pipeline
